In [1]:
#%matplotlib notebook
%matplotlib qt

#This notebook is a testbed for importing PEAC Center USAPI
#raifnall data using pandas and doing some quick analysis

import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as cols
import matplotlib.cm as cm

from mpl_toolkits.basemap import Basemap, shiftgrid

import numpy as np
import numpy.ma as ma

from datetime import date
import datetime

import calendar

from netCDF4 import Dataset

from scipy import signal, linalg, stats

from pycurrents.codas import to_day, to_date
from pycurrents.plot.mpltools import dday_to_mpl

from pycurrents.system import Bunch
from pycurrents.num import eof
from pycurrents.num import rangeslice

import pickle

from scipy.special import comb

In [2]:
datadir = '/home/alejandro/Desktop/Data/'

In [3]:
def extend(dday_1800, dat):
    """
    Return dday, dat extended to fill out the last year.
    
    This function was modifyed to take in only one data variable.
    """
    n_orig = len(dday_1800)
    ymdhms = to_date(1800, dday_1800)
    nmissing = 12 - ymdhms[-1, 1]
    nmonths = n_orig + nmissing
    ymdhms_new = np.zeros((nmonths, 6), dtype=np.uint16)
    ymdhms_new[:n_orig, :] = ymdhms
    ymdhms_new[n_orig:, 0] = ymdhms[-1, 0]
    # Fill in the remaining months:
    ymdhms_new[n_orig:, 1] = np.arange(ymdhms[-1, 1] + 1, 13)
    ymdhms_new[n_orig:, 2] = 1
    dday_new = to_day(1800, ymdhms_new)
    
    shape_new = (nmonths, dat.shape[1], dat.shape[2])
    ## EF: modify to work with masked array or ndarray
    if np.ma.isMA(dat):
        dat_new = np.ma.masked_all(shape_new, float)
    else:    
        dat_new = np.nan + np.zeros(shape_new, float)
    
    dat_new[:n_orig, :, :] = dat
    
    #This Function has been modifyed to also return the date of the last valid data and its ymdhms index
    
    last_valid_date = to_date(1800,dday_1800[-1])
    last_valid_index = n_orig -1
   
    return Bunch(dday_1800=dday_new,
                 ymdhms=ymdhms_new,
                 dat=dat_new,
                 last_valid_date=last_valid_date,
                 last_valid_index=last_valid_index)

def cal_climatology_and_anomaly(data,ymdhms,last_valid_date,last_valid_index,latitudes,longitudes,start_year, end_year):
                           
    #Select the years for climatology from the new ymdhms_padded
    # All the monthly data for the years you want to calculate the climatology
    
    clim_selection = ((ymdhms[:, 0] >= start_year) & (ymdhms[:, 0] <= end_year))
    
    # EF: Trim everything right at the start.
    data = data[clim_selection]
    ymdhms = ymdhms[clim_selection]
    
    #Calculate the total numbers of years in the climatology
    length_in_years = ymdhms[-1, 0] - ymdhms[0, 0] + 1
    
    # EF: ensure we have masked arrays and no nans
    data = np.ma.masked_invalid(data)
    
    #reshape the matrix so that is has dimensions of [years, months, lat, lon]
    reshaped_data = np.reshape(data, (length_in_years, 12, 
                                      len(latitudes), 
                                      len(longitudes)))
    #Calculate the climatology by using nanmeans, along the yeas axis
    #climatology = np.nanmean(reshaped_data, axis=0)
    
    climatology = reshaped_data.mean(axis=0)
    anomaly = reshaped_data - climatology
    anomaly = anomaly.reshape(data.shape)
    
    if end_year < last_valid_date[0]:
        last_valid_date_new = ymdhms[-1,:]
        last_valid_index_new = -1
    else:
        last_valid_date_new = last_valid_date
        last_valid_index_new = np.where((ymdhms[:,0]== last_valid_date[0]) &
                                        (ymdhms[:,1]== last_valid_date[1]) &
                                        (ymdhms[:,2]== last_valid_date[2]))[0][0]
    
    return Bunch(climatology=climatology,
                 anomaly=anomaly, 
                 ymdhms=ymdhms,
                 last_valid_date = last_valid_date_new,
                 last_valid_index = last_valid_index_new)

def seasonal_anomaly(m_anom,last_valid_index):
    s_anom = np.ma.zeros(m_anom.shape, float)
    orig_mask = np.ma.getmaskarray(m_anom)
    
    if last_valid_index == -1:
        s_anom[1:-1] = (m_anom[:-2] + m_anom[1:-1] + m_anom[2:]) / 3
    
        s_anom[0] = (m_anom[0] + m_anom[1]) / 2
        s_anom[-1] = (m_anom[-2] + m_anom[-1]) / 2
    
    else:
        #s_anom[1:last_valid_index-1,:,:] = (m_anom[:last_valid_index-2,:,:] + 
        #                                    m_anom[1:last_valid_index-1,:,:] + 
        #                                    m_anom[2:last_valid_index,:,:]) / 3
        s_anom[1:last_valid_index,:,:] = (m_anom[:last_valid_index-1,:,:] + 
                                          m_anom[1:last_valid_index,:,:] + 
                                          m_anom[2:last_valid_index+1,:,:]) / 3
    
        s_anom[0,:,:] = (m_anom[0,:,:] + m_anom[1,:,:]) / 2
        s_anom[last_valid_index,:,:] = (m_anom[last_valid_index-1,:,:] + 
                                        m_anom[last_valid_index,:,:]) / 2
        
    s_anom_out = np.ma.array(s_anom, mask=orig_mask)
    
    return Bunch(s_anom = s_anom_out, 
                 last_valid_index = last_valid_index)

def seasonal_anomaly2(m_anom, nmin=2):
    newshape = [3] + list(m_anom.shape)
    newshape[1] += 2
    accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
    accum[:] = np.ma.masked
    accum[0, :-2] = m_anom[:]
    accum[1, 1:-1] = m_anom[:]
    accum[2, 2:] = m_anom[:]
    out = accum.mean(axis=0)[1:-1]
    out = np.ma.masked_where(accum[:, 1:-1].count(axis=0) < nmin, out, copy=False)
    return Bunch(s_anom = out)


In [4]:
#sst_seasonal_anom = seasonal_anomaly(sst_ca.anomaly, sst_ca.last_valid_index)
#sst_seasonal_anom2 = seasonal_anomaly2(sst_ca.anomaly, nmin=2)

In [5]:
#print(sst_seasonal_anom.s_anom.shape)
#print(sst_seasonal_anom2.shape)

In [6]:
#np.ma.allclose(sst_seasonal_anom.s_anom, sst_seasonal_anom2, masked_equal = True)

In [7]:
#print(c_PREC_precip_2013)

# Here we read and process the time series data from the excel files

In [8]:
peac_station_rain = pd.ExcelFile(datadir +'PEAC_station_rainfall_database.xlsx')
print(peac_station_rain.sheet_names)

ONI_file = pd.ExcelFile(datadir +'CPC_ONI.xlsx')
print(ONI_file.sheet_names)


['Guam', 'Saipan', 'Koror', 'Yap', 'Chuuk', 'Pohnpei', 'Majuro', 'Kwajalein', 'PagoPago', 'Saipan_old', 'JFM', 'FMA', 'MAM', 'AMJ', 'MJJ', 'JJA', 'JAS', 'ASO', 'SON', 'OND', 'NDJ', 'DJF']
['ONI']


In [9]:
def read_USAPI_data(file_name, island_name):
    
    #Here we read the data into python from the excel file
    raw_data= pd.read_excel(file_name, sheetname = island_name, 
                            skiprows = 1, parse_cols = "A:O")
    
    #
    raw_matrix = raw_data.as_matrix(columns=None)
    print(type(raw_matrix), raw_matrix.dtype)
    
    years =  raw_matrix[:51,1]
    station_id = raw_matrix[0,0]

    rainfall = raw_matrix[:51,3:]

    rainfall[rainfall == '9999'] = np.nan
    rainfall[rainfall == "nan"] = np.nan

    rainfall = rainfall * 0.1
    
    # We have to convert from an object array to floating point.
    rainfall = np.ma.masked_invalid(rainfall.astype(float))
    print('rainfall: ', rainfall.dtype, rainfall[5, 5])
 
    #Initiate the ymdhms array as an array of int values filler with zeros
    #that has the length og the total number of time steps years*months
    ymdhms = np.zeros([51*12,6],dtype = np.int)

    #here we will the first column with the year values from the excel sheet. 
    #repeated each one 12 consecutive times
    ymdhms[:,0] = np.repeat(years,12)

    #Here we create a list of the month indices
    m = np.arange(1,13)
    #we tile the list so that the entire list 1..12 repeats as many times as the number of years
    mm = np.tile(m,51)

    ymdhms[:,1] = mm

    #the day column is filled with 15
    ymdhms[:,2] = 15
    
    dday = to_day(1800, ymdhms)
    mpldays = dday_to_mpl(1800, dday)
    mpldaysformated = mpl.dates.num2date(mpldays)

    out = Bunch(island_name = island_name, station_id = station_id, rainfall=rainfall,
               ymdhms = ymdhms, mpldaysformated = mpldaysformated)
    
    return out
    #return station_id, rainfall, ymdhms, mpldaysformated
    
def read_index_data(file_name, index_name):
    
    #Here we read the data into python from the excel file
    raw_data= pd.read_excel(file_name, sheetname = index_name, skiprows = 1, parse_cols = "A:M")
    
    #
    raw_matrix = raw_data.as_matrix(columns=None)
    print(type(raw_matrix), raw_matrix.dtype)
    
    years =  raw_matrix[:,0]
    index = raw_matrix[:,1:]
    

    # We have to convert from an object array to floating point.
    index = np.ma.masked_invalid(index.astype(float))
    print('index: ', index.dtype, index[5, 5])
 
    #Initiate the ymdhms array as an array of int values filler with zeros
    #that has the length og the total number of time steps years*months
    ymdhms = np.zeros([67*12,6],dtype = np.int)

    #here we will the first column with the year values from the excel sheet. 
    #repeated each one 12 consecutive times
    ymdhms[:,0] = np.repeat(years,12)

    #Here we create a list of the month indices
    m = np.arange(1,13)
    #we tile the list so that the entire list 1..12 repeats as many times as the number of years
    mm = np.tile(m,67)

    ymdhms[:,1] = mm

    #the day column is filled with 15
    ymdhms[:,2] = 15
    
    dday = to_day(1800, ymdhms)
    mpldays = dday_to_mpl(1800, dday)
    mpldaysformated = mpl.dates.num2date(mpldays)

    out = Bunch(index_name = index_name, index = index,
               ymdhms = ymdhms, mpldaysformated = mpldaysformated)
    
    return out
    #return station_id, rainfall, ymdhms, mpldaysformated

def seasonal_anomaly_old(m_anom):
    s_anom = np.ma.zeros(m_anom.shape, float)
    s_anom[1:-1] = (m_anom[:-2] + m_anom[1:-1] + m_anom[2:]) / 3
    s_anom[0] = (m_anom[0] + m_anom[1]) / 2
    s_anom[-1] = (m_anom[-2] + m_anom[-1]) / 2
    return s_anom

def seasonal_sum(m_anom):
    s_anom = np.ma.zeros(m_anom.shape, float)
    s_anom[1:-1] = (m_anom[:-2] + m_anom[1:-1] + m_anom[2:])
    s_anom[0] = (m_anom[0] + m_anom[1])
    s_anom[-1] = (m_anom[-2] + m_anom[-1])
    return s_anom

def running_sum(m_anom, window, nmin = 2):
    
    if window % 2 == 0:
        nmin = (window)/2
        #nmin = 1
        chop = (window-1)

        newshape = [window] + list(m_anom.shape)
        newshape[1] += window -1
        accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
        accum[:] = np.ma.masked
        for i in range(window):
            end = -window+i+1
            if end == 0:
                accum[i, i:] = m_anom[:]
            else:
                accum[i, i:end] = m_anom[:]
        if window != 2:
            start=window/2
            stop = (window/2) -1
            out = accum.mean(axis=0)[start:-stop]
            out = np.ma.masked_where(accum[:,start:-stop].count(axis=0) < nmin, out, copy=False)
        else:
            out = accum.mean(axis=0)[1:]
            out = np.ma.masked_where(accum[:,1:].count(axis=0) < nmin, out, copy=False)
        
    else:
        nmin = (window+1)/2
        chop = (window -1)/2

        newshape = [window] + list(m_anom.shape)
        newshape[1] += window -1
        accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
        accum[:] = np.ma.masked
        for i in range(window):
            end = -window+i+1
            if end == 0:
                accum[i, i:] = m_anom[:]
            else:
                accum[i, i:end] = m_anom[:]
        out = accum.mean(axis=0)[chop:-chop]
        out = np.ma.masked_where(accum[:, chop:-chop].count(axis=0) < nmin, out, copy=False)
        
    return Bunch(s_anom = out)

In [10]:
station_name_list = ["Koror", "Yap", "Guam", "Chuuk", "Pohnpei", "Kwajalein", "Majuro", "Saipan"]
#variable_name_list = [koror, yap, guam, chuuk, phonpei, kwajalein, majuro]

stations = Bunch()
for name in station_name_list:
    stations[name] = read_USAPI_data(peac_station_rain, name)
    

<class 'numpy.ndarray'> object
rainfall:  float64 498.1
<class 'numpy.ndarray'> object
rainfall:  float64 354.3
<class 'numpy.ndarray'> object
rainfall:  float64 128.6
<class 'numpy.ndarray'> object
rainfall:  float64 353.9
<class 'numpy.ndarray'> object
rainfall:  float64 599.9
<class 'numpy.ndarray'> object
rainfall:  float64 292.9
<class 'numpy.ndarray'> object
rainfall:  float64 340.5
<class 'numpy.ndarray'> object
rainfall:  float64 --


In [11]:
for raindata in stations.values():
    #print(raindata.island_name)
    #print(np.shape(raindata.rainfall))
    raindata.monmean = raindata.rainfall.mean(axis=0)
    raindata.monstd = raindata.rainfall.std(axis=0)
    raindata.monanom = raindata.rainfall - raindata.monmean

In [12]:
#print(np.shape(stations.Kwajalein.monanom))
kwajalein_seasonal_anom = seasonal_anomaly_old(stations.Kwajalein.monanom.ravel())
kwajalein_seasonal_total = seasonal_sum(stations.Kwajalein.rainfall.ravel())
kwajalein_dry_season_total = running_sum(stations.Kwajalein.rainfall.ravel(),6)
kwajalein_dry_season_anom = running_sum(stations.Kwajalein.monanom.ravel(),6)

guam_seasonal_anom = seasonal_anomaly_old(stations.Guam.monanom.ravel())
guam_seasonal_total = seasonal_sum(stations.Guam.rainfall.ravel())
guam_dry_season_total = running_sum(stations.Guam.rainfall.ravel(),6)
guam_dry_season_anom = running_sum(stations.Guam.monanom.ravel(),6)


yap_seasonal_anom = seasonal_anomaly_old(stations.Yap.monanom.ravel())
yap_seasonal_total = seasonal_sum(stations.Yap.rainfall.ravel())
yap_dry_season_total = running_sum(stations.Yap.rainfall.ravel(),6)
yap_dry_season_anom = running_sum(stations.Yap.monanom.ravel(),6)

majuro_seasonal_anom = seasonal_anomaly_old(stations.Majuro.monanom.ravel())
majuro_seasonal_total = seasonal_sum(stations.Majuro.rainfall.ravel())
majuro_dry_season_total = running_sum(stations.Majuro.rainfall.ravel(),6)
majuro_dry_season_anom = running_sum(stations.Majuro.monanom.ravel(),6)

chuuk_seasonal_anom = seasonal_anomaly_old(stations.Chuuk.monanom.ravel())
chuuk_seasonal_total = seasonal_sum(stations.Chuuk.rainfall.ravel())
chuuk_dry_season_total = running_sum(stations.Chuuk.rainfall.ravel(),6)
chuuk_dry_season_anom = running_sum(stations.Chuuk.monanom.ravel(),6)

koror_seasonal_anom = seasonal_anomaly_old(stations.Koror.monanom.ravel())
koror_seasonal_total = seasonal_sum(stations.Koror.rainfall.ravel())
koror_dry_season_total = running_sum(stations.Koror.rainfall.ravel(),6)
koror_dry_season_anom = running_sum(stations.Koror.monanom.ravel(),6)

pohnpei_seasonal_anom = seasonal_anomaly_old(stations.Pohnpei.monanom.ravel())
pohnpei_seasonal_total = seasonal_sum(stations.Pohnpei.rainfall.ravel())
pohnpei_dry_season_total = running_sum(stations.Pohnpei.rainfall.ravel(),6)
pohnpei_dry_season_anom = running_sum(stations.Pohnpei.monanom.ravel(),6)

saipan_seasonal_anom = seasonal_anomaly_old(stations.Saipan.monanom.ravel())
saipan_seasonal_total = seasonal_sum(stations.Saipan.rainfall.ravel())
saipan_dry_season_total = running_sum(stations.Saipan.rainfall.ravel(),6)
saipan_dry_season_anom = running_sum(stations.Saipan.monanom.ravel(),6)

/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/ma/core.py:3114: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dout = self.data[indx]
/home/alejandro/anaconda3/lib/python3.5/site-packages/numpy/ma/core.py:3169: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  dout._mask = _mask[indx]


In [13]:
#print(saipan_dry_season_anom.s_anom.reshape)

In [14]:
# window = 6
# m_anom = stations.Pohnpei.rainfall.ravel()

# nmin = (window)/2
# #nmin = 1
# chop = (window-1)

# newshape = [window] + list(m_anom.shape)
# newshape[1] += window -1
# accum = np.ma.zeros(newshape, dtype=m_anom.dtype)
# accum[:] = np.ma.masked
# for i in range(window):
#     end = -window+i+1
#     if end == 0:
#         accum[i, i:] = m_anom[:]
#     else:
#         accum[i, i:end] = m_anom[:]
# if window != 2:
#     start=window/2
#     stop = (window/2) -1
#     out = accum.mean(axis=0)[start:-stop]
#     out = np.ma.masked_where(accum[:,start:-stop].count(axis=0) < nmin, out, copy=False)
# else:
#     out = accum.mean(axis=0)[1:]
#     out = np.ma.masked_where(accum[:,1:].count(axis=0) < nmin, out, copy=False)
    
# print(np.shape(out))
# print(out)

In [15]:
def spi_calculation(data, climatology_series):
    
    good_data = data[~data.mask]
    good_climatology = climatology_series[~climatology_series.mask]
    
    spi = np.empty_like(good_data)
    spi[:] = np.NAN

    #fit gamma distribution to climatology series
    fit_alpha, fit_loc, fit_beta = stats.gamma.fit(good_climatology)

    #find cumulative probabilities of data from fitted distribution
    data_cdf = stats.gamma.cdf(good_data, fit_alpha, loc=fit_loc, scale=fit_beta)

    # find the percent points from the random normal dist

    spi[:] = stats.norm.ppf(data_cdf, loc=0, scale=1)
    
    return spi

In [17]:
# data = saipan_dry_season_total.s_anom.reshape(51, 12)[:,1]
# climatology_series = saipan_dry_season_total.s_anom.reshape(51, 12)[:,1]

# good_data = data[~data.mask]
# good_climatology = climatology_series[~climatology_series.mask]

# spi = np.empty_like(good_data)
# spi[:] = np.NAN

# #fit gamma distribution to climatology series
# fit_alpha, fit_loc, fit_beta = stats.gamma.fit(good_climatology)

# #find cumulative probabilities of data from fitted distribution
# data_cdf = stats.gamma.cdf(good_data, fit_alpha, loc=fit_loc, scale=fit_beta)

# # find the percent points from the random normal dist

# spi[:] = stats.norm.ppf(data_cdf, loc=0, scale=1)

# print(np.shape(spi))
# print(spi)
# print(good_data)
# plt.plot(spi)
# plt.plot(good_data)

In [18]:
kwajalein_dry_season_total_matrix = kwajalein_dry_season_total.s_anom.reshape(51, 12)
kwajalein_dry_season_anom_matrix = kwajalein_dry_season_anom.s_anom.reshape(51, 12)
majuro_dry_season_anom_matrix = majuro_dry_season_anom.s_anom.reshape(51, 12)
guam_dry_season_anom_matrix = guam_dry_season_anom.s_anom.reshape(51, 12)
yap_dry_season_anom_matrix = yap_dry_season_anom.s_anom.reshape(51, 12)
koror_dry_season_anom_matrix = koror_dry_season_anom.s_anom.reshape(51, 12)
chuuk_dry_season_anom_matrix = chuuk_dry_season_anom.s_anom.reshape(51, 12)
pohnpei_dry_season_anom_matrix = pohnpei_dry_season_anom.s_anom.reshape(51, 12)
saipan_dry_season_anom_matrix = saipan_dry_season_anom.s_anom.reshape(51, 12)

#Put ONI data into a pandas table
season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']

seven_month_list = ['Oct-Jan-Apr' , 'Nov-Feb-May' , 'Dec-Mar-Jun',
                    'Jan-Apr-Jul' , 'Feb-May-Aug' , 'Mar-Jun-Sep',
                    'Apr-Jul-Oct' , 'May-Aug-Nov' , 'Jun-Sep-Dec',
                    'Jul-Oct-Jan' , 'Aug-Nov-Feb' , 'Sep-Dec-Mar']   

six_month_list = ['Nov-Apr' , 'Dec-May' , 'Jan-Jun',
                  'Feb-Jul' , 'Mar-Aug' , 'Apr-Sep',
                  'May-Oct' , 'Jun-Nov' , 'Jul-Dec',
                  'Aug-Jan' , 'Sep-Feb' , 'Oct-Mar']    

kawj_dry_season_total_df = pd.DataFrame(kwajalein_dry_season_total_matrix, columns = six_month_list, index = range(1966,2017))
kawj_dry_season_anom_df = pd.DataFrame(kwajalein_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))
#kawj_df

In [19]:
kwajalein_dry_season_total_matrix = kwajalein_dry_season_total.s_anom.reshape(51, 12)

kwajalein_spi_matrix = np.empty_like(kwajalein_dry_season_total_matrix)
kwajalein_spi_matrix[:] = np.NAN

for col in range(kwajalein_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(kwajalein_dry_season_total_matrix[:,col]),
                          np.squeeze(kwajalein_dry_season_total_matrix[:,col]))

    kwajalein_spi_matrix[:len(spi),col] = spi
    

    
guam_dry_season_total_matrix = guam_dry_season_total.s_anom.reshape(51, 12)
guam_spi_matrix = np.empty_like(guam_dry_season_total_matrix)
guam_spi_matrix[:] = np.NAN

for col in range(guam_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(guam_dry_season_total_matrix[:,col]),
                          np.squeeze(guam_dry_season_total_matrix[:,col]))

    guam_spi_matrix[:len(spi),col] = spi
    
    
yap_dry_season_total_matrix = yap_dry_season_total.s_anom.reshape(51, 12)
yap_spi_matrix = np.empty_like(yap_dry_season_total_matrix)
yap_spi_matrix[:] = np.NAN

for col in range(yap_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(yap_dry_season_total_matrix[:,col]),
                          np.squeeze(yap_dry_season_total_matrix[:,col]))

    yap_spi_matrix[:len(spi),col] = spi
    
majuro_dry_season_total_matrix = majuro_dry_season_total.s_anom.reshape(51, 12)
majuro_spi_matrix = np.empty_like(majuro_dry_season_total_matrix)
majuro_spi_matrix[:] = np.NAN

for col in range(majuro_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(majuro_dry_season_total_matrix[:,col]),
                          np.squeeze(majuro_dry_season_total_matrix[:,col]))

    majuro_spi_matrix[:len(spi),col] = spi
    
    
koror_dry_season_total_matrix = koror_dry_season_total.s_anom.reshape(51, 12)
koror_spi_matrix = np.empty_like(koror_dry_season_total_matrix)
koror_spi_matrix[:] = np.NAN

for col in range(koror_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(koror_dry_season_total_matrix[:,col]),
                          np.squeeze(koror_dry_season_total_matrix[:,col]))

    koror_spi_matrix[:len(spi),col] = spi
    
    
chuuk_dry_season_total_matrix = chuuk_dry_season_total.s_anom.reshape(51, 12)
chuuk_spi_matrix = np.empty_like(chuuk_dry_season_total_matrix)
chuuk_spi_matrix[:] = np.NAN

for col in range(chuuk_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(chuuk_dry_season_total_matrix[:,col]),
                          np.squeeze(chuuk_dry_season_total_matrix[:,col]))

    chuuk_spi_matrix[:len(spi),col] = spi
    
    
pohnpei_dry_season_total_matrix = pohnpei_dry_season_total.s_anom.reshape(51, 12)
pohnpei_spi_matrix = np.empty_like(pohnpei_dry_season_total_matrix)
pohnpei_spi_matrix[:] = np.NAN

for col in range(pohnpei_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(pohnpei_dry_season_total_matrix[:,col]),
                          np.squeeze(pohnpei_dry_season_total_matrix[:,col]))

    pohnpei_spi_matrix[:len(spi),col] = spi
    
saipan_dry_season_total_matrix = saipan_dry_season_total.s_anom.reshape(51, 12)
saipan_spi_matrix = np.empty_like(saipan_dry_season_total_matrix)
saipan_spi_matrix[:] = np.NAN
saipan_spi_matrix_temp = np.empty_like(saipan_dry_season_total_matrix)
saipan_spi_matrix_temp[:] = np.NAN

for col in range(saipan_dry_season_total_matrix.shape[1]):
    #print(kwajalein_seasonal_total_matrix[:,col])
    spi = spi_calculation(np.squeeze(saipan_dry_season_total_matrix[:,col]),
                          np.squeeze(saipan_dry_season_total_matrix[:,col]))

    saipan_spi_matrix_temp[-len(spi):,col] = spi

saipan_spi_matrix_temp[saipan_spi_matrix_temp == np.inf] = 7
saipan_mask = np.ma.getmask(saipan_dry_season_total_matrix)
saipan_spi_matrix = np.ma.masked_invalid(np.ma.array(saipan_spi_matrix_temp,mask = saipan_mask))
    
#Put ONI data into a pandas table
seven_month_list = ['Oct-Jan-Apr' , 'Nov-Feb-May' , 'Dec-Mar-Jun',
                    'Jan-Apr-Jul' , 'Feb-May-Aug' , 'Mar-Jun-Sep',
                    'Apr-Jul-Oct' , 'May-Aug-Nov' , 'Jun-Sep-Dec',
                    'Jul-Oct-Jan' , 'Aug-Nov-Feb' , 'Sep-Dec-Mar']    
    
six_month_list = ['Nov-Apr' , 'Dec-May' , 'Jan-Jun',
                  'Feb-Jul' , 'Mar-Aug' , 'Apr-Sep',
                  'May-Oct' , 'Jun-Nov' , 'Jul-Dec',
                  'Aug-Jan' , 'Sep-Feb' , 'Oct-Mar']    

In [20]:
# saipan_spi_matrix[saipan_spi_matrix == np.inf] = 10
# saipan_mask = np.ma.getmask(saipan_dry_season_total_matrix)
# saipan_spi_matrix_2 = np.ma.masked_invalid(np.ma.array(saipan_spi_matrix,mask = saipan_mask))
# print(saipan_dry_season_total_matrix)
# print(saipan_spi_matrix_2)


In [21]:
# saipan_dry_season_total_matrix = saipan_dry_season_total.s_anom.reshape(51, 12)[:,1]

# good_data = saipan_dry_season_total_matrix[~saipan_dry_season_total_matrix.mask]
# good_climatology = saipan_dry_season_total_matrix[~saipan_dry_season_total_matrix.mask]

# spi = np.empty_like(good_data)
# spi[:] = np.NAN

# #fit gamma distribution to climatology series
# fit_alpha, fit_loc, fit_beta = stats.gamma.fit(good_climatology)

# #find cumulative probabilities of data from fitted distribution
# data_cdf = stats.gamma.cdf(good_data, fit_alpha, loc=fit_loc, scale=fit_beta)

# # find the percent points from the random normal dist

# spi[:] = stats.norm.ppf(data_cdf, loc=0, scale=1)
# print(saipan_dry_season_total.s_anom.reshape(51, 12)[:,1])
# print(spi)

In [22]:
# saipan_dry_season_total_matrix = saipan_dry_season_total.s_anom.reshape(51, 12)
# saipan_spi_matrix = np.empty_like(saipan_dry_season_total_matrix)
# saipan_spi_matrix[:] = np.NAN

# for col in range(saipan_dry_season_total_matrix.shape[1]):
#     #print(kwajalein_seasonal_total_matrix[:,col])
#     spi = spi_calculation(np.squeeze(saipan_dry_season_total_matrix[:,col]),
#                           np.squeeze(saipan_dry_season_total_matrix[:,col]))

#     saipan_spi_matrix[-len(spi):,col] = spi
#     #test[:len(spi),col] = spi
    
# #print(np.ma.masked_invalid(saipan_spi_matrix))
# print(saipan_dry_season_total_matrix)


In [23]:
def spi_d_category_cells_dry(data):
    
    if -0.5 >= data > -0.8:
        color = 'yellow'
    elif -0.8 >= data > -1.3:
        color = 'sandybrown'
    elif -1.3 >= data > -1.5:
        color = 'darkorange'
    elif -1.5 >= data > -1.9:
        color = 'red'
    elif -2 >= data:
        color = 'darkred'
    else:
        color = ''
        #attr = 'background-color: {}'.format('yellow')
    return 'background-color: %s' %color

def spi_d_category_cells_wet(data):
    
    if 0.5 <= data:
        color = 'skyblue'
    else:
        color = ''
        #attr = 'background-color: {}'.format('yellow')
    return 'background-color: %s' %color

In [24]:
kwajalein_SPI_df = pd.DataFrame(kwajalein_spi_matrix, columns = six_month_list, index = range(1966,2017))
majuro_SPI_df = pd.DataFrame(majuro_spi_matrix, columns = six_month_list, index = range(1966,2017))
guam_SPI_df = pd.DataFrame(guam_spi_matrix, columns = six_month_list, index = range(1966,2017))


In [25]:
saipan_SPI_df = pd.DataFrame(saipan_spi_matrix, columns = six_month_list, index = range(1966,2017))
saip_dry_season_anom_df = pd.DataFrame(saipan_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))
#saipan_SPI_df

In [26]:
oni = read_index_data(ONI_file, "ONI")
oni_selection = ((oni.ymdhms[:, 0] >= 1966))
oni_time_series = oni.index.ravel()
oni_period = oni_time_series[oni_selection]
oni_period_matrix = oni_period.reshape(51,12)
ONI_df = pd.DataFrame(oni_period_matrix, columns = season_list, index = range(1966,2017))

<class 'numpy.ndarray'> float64
index:  float64 -0.6


# Calculate the agreggate drought for 
## Kwajalein, Majuro, Yap and Guam

## First using the average SPI for the 4 stations

In [27]:
average_spi_matrix = np.mean(np.array([kwajalein_spi_matrix, majuro_spi_matrix, guam_spi_matrix, yap_spi_matrix]), axis=0)


average_dry_season_anom_matrix = np.mean(np.array([kwajalein_dry_season_anom_matrix, 
                                                   majuro_dry_season_anom_matrix, 
                                                   guam_dry_season_anom_matrix, 
                                                   yap_dry_season_anom_matrix]), axis=0)

average_all_station_spi_matrix = np.mean(np.array([kwajalein_spi_matrix, 
                                                   majuro_spi_matrix, 
                                                   guam_spi_matrix, 
                                                   yap_spi_matrix,
                                                   chuuk_spi_matrix,
                                                   koror_spi_matrix,
                                                   pohnpei_spi_matrix]), axis=0)


average_all_station_dry_season_anom_matrix = np.mean(np.array([kwajalein_dry_season_anom_matrix, 
                                                               majuro_dry_season_anom_matrix, 
                                                               guam_dry_season_anom_matrix, 
                                                               yap_dry_season_anom_matrix,
                                                               chuuk_dry_season_anom_matrix, 
                                                               koror_dry_season_anom_matrix, 
                                                               pohnpei_dry_season_anom_matrix,]), axis=0)

average_6_station_spi_matrix = np.mean(np.array([kwajalein_spi_matrix, 
                                                   majuro_spi_matrix, 
                                                   guam_spi_matrix, 
                                                   yap_spi_matrix,
                                                   pohnpei_spi_matrix,
                                                   koror_spi_matrix]), axis=0)


average_6_station_dry_season_anom_matrix = np.mean(np.array([kwajalein_dry_season_anom_matrix, 
                                                               majuro_dry_season_anom_matrix, 
                                                               guam_dry_season_anom_matrix, 
                                                               yap_dry_season_anom_matrix,
                                                               pohnpei_dry_season_anom_matrix, 
                                                               koror_dry_season_anom_matrix,]), axis=0)

average_5_station_spi_matrix = np.mean(np.array([kwajalein_spi_matrix, 
                                                   majuro_spi_matrix, 
                                                   guam_spi_matrix, 
                                                   yap_spi_matrix,
                                                   chuuk_spi_matrix]), axis=0)


average_5_station_dry_season_anom_matrix = np.mean(np.array([kwajalein_dry_season_anom_matrix, 
                                                               majuro_dry_season_anom_matrix, 
                                                               guam_dry_season_anom_matrix, 
                                                               yap_dry_season_anom_matrix,
                                                               chuuk_dry_season_anom_matrix]), axis=0)

average_3_station_spi_matrix = np.mean(np.array([kwajalein_spi_matrix, 
                                                   guam_spi_matrix, 
                                                   yap_spi_matrix]), axis=0)


average_3_station_dry_season_anom_matrix = np.mean(np.array([kwajalein_dry_season_anom_matrix, 
                                                               guam_dry_season_anom_matrix, 
                                                               yap_dry_season_anom_matrix]), axis=0)

average_2_station_spi_matrix = np.mean(np.array([kwajalein_spi_matrix, 
                                                   guam_spi_matrix]), axis=0)


average_2_station_dry_season_anom_matrix = np.mean(np.array([kwajalein_dry_season_anom_matrix, 
                                                               guam_dry_season_anom_matrix]), axis=0)

average_3_saip_station_spi_matrix = np.nanmean(np.array([kwajalein_spi_matrix,
                                                      saipan_spi_matrix,
                                                      guam_spi_matrix]), axis=0)


average_3_saip_station_dry_season_anom_matrix = np.nanmean(np.array([kwajalein_dry_season_anom_matrix,
                                                                     saipan_dry_season_anom_matrix,
                                                                     guam_dry_season_anom_matrix]), axis=0)

average_SPI_df = pd.DataFrame(average_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_dry_season_anom_df = pd.DataFrame(average_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))

average_all_station_SPI_df = pd.DataFrame(average_all_station_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_all_station_dry_season_anom_df = pd.DataFrame(average_all_station_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))

average_6_station_SPI_df = pd.DataFrame(average_6_station_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_6_station_dry_season_anom_df = pd.DataFrame(average_6_station_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))

average_5_station_SPI_df = pd.DataFrame(average_5_station_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_5_station_dry_season_anom_df = pd.DataFrame(average_5_station_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))

average_3_station_SPI_df = pd.DataFrame(average_3_station_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_3_station_dry_season_anom_df = pd.DataFrame(average_3_station_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))

average_2_station_SPI_df = pd.DataFrame(average_2_station_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_2_station_dry_season_anom_df = pd.DataFrame(average_2_station_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))

average_3_saip_station_SPI_df = pd.DataFrame(average_3_saip_station_spi_matrix, columns = six_month_list, index = range(1966,2017))
average_3_saip_station_dry_season_anom_df = pd.DataFrame(average_3_saip_station_dry_season_anom_matrix, columns = six_month_list, index = range(1966,2017))


In [27]:
from sklearn.cluster import KMeans

kwaj = stations.Kwajalein.monmean
guam = stations.Guam.monmean
majuro = stations.Majuro.monmean
yap = stations.Yap.monmean
koror = stations.Koror.monmean
chuuk = stations.Chuuk.monmean
pohnpei = stations.Pohnpei.monmean

stations_mon_means = np.array([kwaj, guam, majuro, yap, koror, chuuk, pohnpei])

print(np.shape(stations_mon_means))

s_kmeans = KMeans(n_clusters=3).fit(stations_mon_means)
print(s_kmeans.labels_)
print(s_kmeans.cluster_centers_)

(7, 12)
[0 0 1 0 1 1 2]
[[ 146.41011765  113.43419608  116.57205229  143.59424837  184.69294118
   236.18760784  315.91762667  354.27481333  336.5306      310.58204
   240.06989333  192.74466333]
 [ 232.78579085  199.32577778  219.1208366   270.04049673  298.55929412
   343.23721569  358.93702667  333.48104     319.41770667  327.40458667
   298.14862667  294.5678    ]
 [ 304.41062745  256.22133333  360.9854902   436.92576471  494.87407843
   423.47772549  425.68704     395.9138      381.44144     392.94344
   388.62536     386.81464   ]]


# Time Series plots

In [29]:
# six_month_list = ['Nov-Apr' , 'Dec-May' , 'Jan-Jun',
#                   'Feb-Jul' , 'Mar-Aug' , 'Apr-Sep',
#                   'May-Oct' , 'Jun-Nov' , 'Jul-Dec',
#                   'Aug-Jan' , 'Sep-Feb' , 'Oct-Mar']  

oni_selection = (np.logical_and((oni.ymdhms[:, 0] >= 1966), (oni.ymdhms[:, 1]== 2)))
ymdhms_dates = oni.ymdhms[oni_selection]

mpl_dates = np.empty(len(ymdhms_dates), dtype='datetime64[M]')

for i, ymdhms in enumerate(ymdhms_dates):
    date = ymdhms_dates[i]
    mpl_dates[i] = datetime.date(date[0],date[1],date[2])


varlist = [saip_dry_season_anom_df['Dec-May'], 
           saipan_SPI_df['Dec-May'], 
           average_3_saip_station_dry_season_anom_df['Dec-May'], 
           average_3_saip_station_SPI_df['Dec-May']]
varlist_2 = [saipan_SPI_df['Dec-May'], 
             ONI_df['DJF'], 
             average_3_station_SPI_df['Dec-May'], 
             ONI_df['DJF']]
plot_titles = ['Saipan Dry Season Rainfall Anom and Dry Season SPI',
              'Saipan Dry Season SPI and ONI',
              'Yap, Guam and Kwaj, Dry Season Rainfall Anom and Dry Season SPI',
              'Yap, Guam and Kwaj, Dry Season SPI and ONI']
ylab = ['(mm/month)', 'SPI', '(mm/month)', 'SPI']
ylab_2 = ['SPI', 'ONI (C)', 'SPI', 'ONI (c)']

fig, axs = plt.subplots(nrows=4, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(6.5, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for var, var2, tit, ax, y, y2 in zip(varlist,varlist_2, plot_titles, axs, ylab, ylab_2):
    cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit+cor, fontsize = 8)
    lines = ax.plot(mpl_dates, var, 'b-')
    ax.set_ylabel(y, color='b')
    ax.grid(True)
    
    ax_max = np.amax(var)
    ax_min = np.amin(var)
    ax_tot_max = np.maximum(ax_max, np.absolute(ax_min))
    ax.set_ylim([-ax_tot_max,ax_tot_max])
    
    ax_e = ax.twinx()
    line_2 = ax_e.plot(mpl_dates, var2, 'r-')
    ax_e.set_ylabel(y2, color='r')
    
    ax_max = np.amax(var2)
    ax_min = np.amin(var2)
    ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
    ax_e.set_ylim([-ax_tot_max2,ax_tot_max2])
    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)
    #lines_list.append(lines)
fig.savefig('saipan_time_series.pdf')

In [39]:
varlist = [average_SPI_df.loc[1979:]['Dec-May'],  
           average_3_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Yap, Guam, Maj and Kwaj, Avg Dry Season SPI vs ONI',
               'Yap, Guam and Kwaj Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1979-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1979,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
    plt.legend(loc='upper center')
    
    ax_max = np.amax(xvar)
    ax_min = np.amin(xvar)
    ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
    
    if y == 'SPI':
        ax.set_ylim([-2,2])
    else:
        ax_max = np.amax(yvar)
        ax_min = np.amin(yvar)
        ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
        ax.set_ylim([-ax_tot_max2-10,ax_tot_max2+10])
        
    #if ax ==1:

    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)

#plt.legend(bbox_to_anchor = (0,0),loc = 'upper left',borderaxespad =0.)

plt.legend()
    
fig.savefig('scatterplots_YGMK_SPI_v_YGK_SPI.pdf')

/home/alejandro/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [ ]:
varlist = [average_SPI_df.loc[1979:]['Dec-May'],  
           average_3_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Yap, Guam, Maj and Kwaj, Avg Dry Season SPI vs ONI',
               'Yap, Guam and Kwaj Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1979-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1979,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
    plt.legend(loc='upper center')
    
    ax_max = np.amax(xvar)
    ax_min = np.amin(xvar)
    ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
    
    if y == 'SPI':
        ax.set_ylim([-2,2])
    else:
        ax_max = np.amax(yvar)
        ax_min = np.amin(yvar)
        ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
        ax.set_ylim([-ax_tot_max2-10,ax_tot_max2+10])
        
    #if ax ==1:

    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)

#plt.legend(bbox_to_anchor = (0,0),loc = 'upper left',borderaxespad =0.)

plt.legend()
    
fig.savefig('scatterplots_YGMK_SPI_v_YGK_SPI.pdf')

In [28]:
varlist = [average_2_station_SPI_df.loc[1979:]['Dec-May'],  
           average_all_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Guam and Kwaj, Avg Dry Season SPI vs ONI',
               'All Station Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1979-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1979,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
    plt.legend(loc='upper center')
    
    ax_max = np.amax(xvar)
    ax_min = np.amin(xvar)
    ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
    
    if y == 'SPI':
        ax.set_ylim([-2,2])
    else:
        ax_max = np.amax(yvar)
        ax_min = np.amin(yvar)
        ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
        ax.set_ylim([-ax_tot_max2-10,ax_tot_max2+10])
        
    #if ax ==1:

    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)

#plt.legend(bbox_to_anchor = (0,0),loc = 'upper left',borderaxespad =0.)

plt.legend()
    
fig.savefig('scatterplots_GK_SPI_v_ALL_SPI.pdf')

/home/alejandro/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [42]:
varlist = [average_2_station_SPI_df.loc[1979:]['Dec-May'],  
           average_3_saip_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Guam, and Kwaj, Avg Dry Season SPI vs ONI',
               'Guam, Kwaj ans Saip Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1979-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1979,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
    plt.legend(loc='upper center')
    
    ax_max = np.amax(xvar)
    ax_min = np.amin(xvar)
    ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
    
    if y == 'SPI':
        ax.set_ylim([-2,2])
    else:
        ax_max = np.amax(yvar)
        ax_min = np.amin(yvar)
        ax_tot_max2 = np.maximum(ax_max, np.absolute(ax_min))
        ax.set_ylim([-ax_tot_max2-10,ax_tot_max2+10])
        
    year_labs = [str(x) for x in list(np.arange(1979,2017))]
    
    for year, x, y in zip(year_labs, xvar, yvar):
        ax.annotate(year,xy=(x,y), xytext = (-20,0),
                   textcoords = 'offset points',
                   #textcoords = 'figure fraction', 
                   ha = 'right', va='bottom', fontsize = 10,
                   arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)

#plt.legend(bbox_to_anchor = (0,0),loc = 'upper left',borderaxespad =0.)

plt.legend()
    
fig.savefig('scatterplots_GK_SPI_v_GKS_SPI_labs.pdf')

/home/alejandro/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [31]:
# seven_month_list = ['Oct-Jan-Apr' , 'Nov-Feb-May' , 'Dec-Mar-Jun',
#                     'Jan-Apr-Jul' , 'Feb-May-Aug' , 'Mar-Jun-Sep',
#                     'Apr-Jul-Oct' , 'May-Aug-Nov' , 'Jun-Sep-Dec',
#                     'Jul-Oct-Jan' , 'Aug-Nov-Feb' , 'Sep-Dec-Mar']    


varlist = [average_3_station_SPI_df.loc[1979:]['Dec-May'],  
           average_2_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Yap, Guam, and Kwaj, Avg Dry Season SPI vs ONI',
               'Guam and Kwaj Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1966-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1966,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
    year_labs = [str(x) for x in list(np.arange(1979,2017))]
    
    for year, x, y in zip(year_labs, xvar, yvar):
        ax.annotate(year,xy=(x,y), xytext = (-20,0),
                   textcoords = 'offset points',
                   #textcoords = 'figure fraction', 
                   ha = 'right', va='bottom', fontsize = 10,
                   arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
    
    plt.legend()

    ax.set_ylim([-3,3])
    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)
fig.savefig('scatterplots_3_station_SPI_v_2_station_SPI_labs.pdf')

/home/alejandro/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [34]:
# seven_month_list = ['Oct-Jan-Apr' , 'Nov-Feb-May' , 'Dec-Mar-Jun',
#                     'Jan-Apr-Jul' , 'Feb-May-Aug' , 'Mar-Jun-Sep',
#                     'Apr-Jul-Oct' , 'May-Aug-Nov' , 'Jun-Sep-Dec',
#                     'Jul-Oct-Jan' , 'Aug-Nov-Feb' , 'Sep-Dec-Mar']    


varlist = [average_3_station_SPI_df.loc[1979:]['Dec-May'],  
           average_3_saip_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Yap, Guam, and Kwaj, Avg Dry Season SPI vs ONI',
               'Guam, Saip, and Kwaj Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1966-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1966,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
    year_labs = [str(x) for x in list(np.arange(1979,2017))]
    
    for year, x, y in zip(year_labs, xvar, yvar):
        ax.annotate(year,xy=(x,y), xytext = (-20,0),
                   textcoords = 'offset points',
                   #textcoords = 'figure fraction', 
                   ha = 'right', va='bottom', fontsize = 10,
                   arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
    
    plt.legend()

    ax.set_ylim([-3,3])
    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)
fig.savefig('scatterplots_KGY_station_SPI_v_3_KGS_station_SPI_labs.pdf')

/home/alejandro/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


In [203]:
varlist = [average_3_station_SPI_df.loc[1979:]['Dec-May'],  
           average_3_saip_station_SPI_df.loc[1979:]['Dec-May']]
varlist_2 = [ONI_df.loc[1979:]['DJF'], 
             ONI_df.loc[1979:]['DJF']]
plot_titles = ['Yap, Guam, and Kwaj, Avg Dry Season SPI vs ONI',
               'Yap and Guam Avg Dry Season SPI vs ONI']
ylab = ['SPI', 'SPI']
xlab = ['ONI (C)', 'ONI (c)']

fig, axs = plt.subplots(ncols=2, sharex=True, 
                        tight_layout=True,  # trims margins
                        figsize=(12, 8))

fname = 'Time'
#axs[0].set_title(fname)
#lines_list = [] we dont really use this
for yvar, xvar, tit, ax, y, x in zip(varlist,varlist_2, plot_titles, axs, ylab, xlab):
    #cor = ' Corr=' + str('%.4f' % round(np.corrcoef(var,var2)[0][1],4))
    ax.set_title(tit, fontsize = 12)
    scatter = ax.scatter(xvar, yvar, c='k', label = '1966-2016')
    ax.set_ylabel(y, color='k')
    ax.set_ylim(-3,3)
    ax.set_xlim(-2.5,2.5)
    ax.set_xlabel(x, color='k')
    ax.grid(True)
    
    year_list = [ 1984, 2001, 2006, 2009, 2013]
    dry_y = np.empty(len(year_list))
    dry_x = np.empty(len(year_list))
    non_dry_year_list = [1985, 1986, 1996, 1997, 2014]
    non_dry_y = np.empty(len(non_dry_year_list))
    non_dry_x = np.empty(len(non_dry_year_list))
    
    la_nina_list =  [1989, 1999, 2000, 2008, 2011]
    lanina_y = np.empty(len(la_nina_list))
    lanina_x = np.empty(len(la_nina_list))
    
    for year in range(1966,2017):
        
        if year in year_list:
            i = year_list.index(year)
            dry_y[i] = yvar[year]
            dry_x[i] = xvar[year]
            #scatter = ax.scatter(xvar[year], yvar[year], c='r', label = 'Members of Dry Composite')
         
        elif year in la_nina_list:
            i = la_nina_list.index(year)
            lanina_y[i] = yvar[year]
            lanina_x[i] = xvar[year]
            
        elif year in non_dry_year_list:
            i = non_dry_year_list.index(year)
            non_dry_y[i] = yvar[year]
            non_dry_x[i] = xvar[year]
            
    scatter = ax.scatter(dry_x, dry_y, c='r', label = 'Members of Dry Composite')
    scatter = ax.scatter(non_dry_x, non_dry_y, c='deepskyblue', label = 'Members of Non Dry Composite')
    scatter = ax.scatter(lanina_x, lanina_y, c='b', label = 'Members of La Nina Composite')
    
#     year_labs = [str(x) for x in list(np.arange(1979,2017))]
    
#     for year, x, y in zip(year_labs, xvar, yvar):
#         ax.annotate(year,xy=(x,y), xytext = (-20,0),
#                    textcoords = 'offset points',
#                    #textcoords = 'figure fraction', 
#                    ha = 'right', va='bottom', fontsize = 10,
#                    arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))
    
    plt.legend()

    ax.set_ylim([-3,3])
    
    # Reduce the number of y-axis ticks:
    ax.locator_params(axis='y', nbins=4)
fig.savefig('scatterplots_3_station_SPI_v_3_saip_station_SPI.pdf')

/home/alejandro/anaconda3/lib/python3.5/site-packages/matplotlib/axes/_axes.py:519: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


## stdev of the kwajalein rainfall
Now we must calculate the stdev of the seasonal anomaly. The most straight forward way to calculate this may be unraveling the monthly anomaly into a series, using the seasonal_anomaly function and reshaping the series back into a matrix, then calculate the stdev of the columns of that matrix.

In [ ]:
#here we restructure the seaqsonal anomaly time series into matices for easier logical selection
kwajalein_seasonal_anom_matrix = kwajalein_seasonal_anom.reshape(51, 12)
kwajalein_seasonal_total_matrix = kwajalein_seasonal_total.reshape(51, 12)
oni_period_matrix = oni_period.reshape(51,12)

#Here we calculate the stdev of eac column of the matrix this is the stdev of seasonal rainfall anomaly for each season
kwajalein_season_std = kwajalein_seasonal_anom_matrix.std(axis=0)
#we repeat the row 51 times to have a matrix the same size as the data matrix and the oni matrix
kwajalein_season_std_matrix = np.tile(kwajalein_season_std,(51,1))


# Aplication of conditions for composite membership

## first for the rainfall and ONI conditions only

Here we apply the condition for dry and very dry seasons

Very dry is considered less than 4 inches a month, 12 inches a season or 305mm

Dry is considered less than 8 inches a month, 24 inches a season or 610mm

In [ ]:
cond_1stdev = kwajalein_seasonal_total_matrix <= 305
cond_oni = oni_period_matrix <= -0.1
cond_verydry_cool = np.logical_and(cond_1stdev, cond_oni)

cond_05stdev = kwajalein_seasonal_total_matrix <= 610
cond_oni = oni_period_matrix <= -0.1
cond_dry_cool = np.logical_and(cond_05stdev, cond_oni)

## nos for the pattern correlation conditions too

### we use only the 90% test level

In [ ]:
#cond_verydry_cool_90patcor = np.logical_and.reduce((cond_1stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev))
#cond_dry_cool_90patcor = np.logical_and.reduce((cond_05stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev))

cond_verydry_cool_90patcor = np.logical_and(cond_verydry_cool[:-1,:], pat_cor_90_test_lev)
cond_dry_cool_90patcor = np.logical_and(cond_dry_cool[:-1,:], pat_cor_90_test_lev)

#cond_verydry_cool_90patcor = np.logical_and(cond_1stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev)
#cond_dry_cool_90patcor = np.logical_and(cond_05stdev[:-1,:], cond_oni[:-1,:], pat_cor_90_test_lev)

# Create the years selected tables

In [ ]:
#Put ONI data into a pandas table
season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']

def oni_color(val):
    
    if val >= 0.5:
        color = 'red'
    elif val <= -0.5 :
        color = 'blue'
    else:
        color = 'black'
    return 'color: %s' % color



ONI_df_color = ONI_df.style.applymap(oni_color)

ONI_df_color

In [ ]:
def color_cells_dry(data, color = 'yellow', axis = None):
    
    attr = 'background-color: {}'.format(color)
    
    return pd.DataFrame(np.where(cond_dry_cool[:-1,:], attr, ''),
                       index = data.index,
                       columns = data.columns)

def color_cells_verydry(data, color = 'darkorange', axis = None):
    
    attr = 'background-color: {}'.format(color)
    
    return pd.DataFrame(np.where(cond_verydry_cool[:-1,:], attr, ''),
                       index = data.index,
                       columns = data.columns)


ONI_df_color_back = ONI_df.style.\
                                apply(color_cells_dry, axis = None).\
                                apply(color_cells_verydry, axis = None)
        
ONI_df_color_back

In [ ]:
def color_cells_dry(data, color = 'yellow', axis = None):
    
    attr = 'background-color: {}'.format(color)
    
    return pd.DataFrame(np.where(cond_dry_cool[:-1,:], attr, ''),
                       index = data.index,
                       columns = data.columns)

def color_cells_verydry(data, color = 'darkorange', axis = None):
    
    attr = 'background-color: {}'.format(color)
    
    return pd.DataFrame(np.where(cond_verydry_cool[:-1,:], attr, ''),
                       index = data.index,
                       columns = data.columns)


Kwaj_seasonal_total_rain_df = pd.DataFrame(kwajalein_seasonal_total_matrix[:-1,:], columns = season_list, index = range(1966,2016))

Kwaj_color_back = Kwaj_seasonal_total_rain_df.style.\
                                apply(color_cells_dry, axis = None).\
                                apply(color_cells_verydry, axis = None)
        
Kwaj_color_back

In [ ]:
Kwaj_mon_rain_df = pd.DataFrame(stations.Kwajalein.rainfall[:-1,:], columns = season_list, index = range(1966,2016))

Kwaj_color_back = Kwaj_mon_rain_df.style.\
                                apply(color_cells_dry, axis = None).\
                                apply(color_cells_verydry, axis = None)
        
Kwaj_color_back

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages

pp = PdfPages('Total_rainfall_Dry_and_VeryDry_seasons.pdf')

# Calculate some sizes for formatting - constants are arbitrary - play around
nrows, ncols = len(ONI_df)+1, len(ONI_df.columns) + 10
hcell, wcell = 0.2, 0.4
hpad, wpad = 0.05, 0   

#put the table on a correctly sized figure    
fig=plt.figure(figsize=(ncols*wcell+wpad, nrows*hcell+hpad))
plt.gca().axis('off')

#print_out_cell_colours = np.where(cond_dry_cool[:-1,:], 'yellow', 'white')
print_out_cell_colours = np.where(cond_dry_cool[:-1,:], 'yellow', 'white')
print_out_cell_colours = np.where(cond_verydry_cool[:-1,:], 'darkorange', print_out_cell_colours )

matplotlib_tab = pd.tools.plotting.table(plt.gca(),ONI_df, 
                                         cellColours = print_out_cell_colours,
                                         loc='center')

plt.suptitle("Total Seasonal Rainfall")

pp.savefig()
plt.close()

pp.close()

In [ ]:
pp = PdfPages('Total_rainfall_Dry_and_VeryDry_90patcor_seasons.pdf')

# Calculate some sizes for formatting - constants are arbitrary - play around
nrows, ncols = len(ONI_df)+1, len(ONI_df.columns) + 10
hcell, wcell = 0.2, 0.4
hpad, wpad = 0.05, 0   

#put the table on a correctly sized figure    
fig=plt.figure(figsize=(ncols*wcell+wpad, nrows*hcell+hpad))
plt.gca().axis('off')

#print_out_cell_colours = np.where(cond_dry_cool[:-1,:], 'yellow', 'white')
print_out_cell_colours = np.where(cond_dry_cool_90patcor, 'yellow', 'white')
print_out_cell_colours = np.where(cond_verydry_cool_90patcor, 'darkorange', print_out_cell_colours )

matplotlib_tab = pd.tools.plotting.table(plt.gca(),ONI_df, 
                                         cellColours = print_out_cell_colours,
                                         loc='center')    

plt.suptitle("Total Seasonal Rainfall and 90 patcor")

pp.savefig()
plt.close()

pp.close()

# Now for the maps of individual years

## Define plotting functions

In [ ]:
def plotting_function(color_field, contour_field, u_wnd, v_wnd, 
                          color_field_name, case_name, target_dir, *argv, save = 'yes'):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.015, top=0.94,
                         wspace=0.05, hspace=0.001)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    mm = range(9)

    for mon, pax in zip(mm, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax

        #reverse the coolwarm palatte to make blue rainy and red dry
        cmap = plt.get_cmap('BrBG')
        bounds = [-2.5,-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5]


        x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
        im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                        extend='both')

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')


        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        
        if box_coords in argv:
            x1,y1 = m(box_coords[2], box_coords[1])
            x2,y2 = m(box_coords[2], box_coords[0])
            x3,y3 = m(box_coords[3], box_coords[0])
            x4,y4 = m(box_coords[3], box_coords[1])
            
            p = Polygon([(x1,y1), (x2,y2), (x3,y3), (x4,y4)], 
                        edgecolor='b', facecolor = 'none', linewidth = 2)
            m.ax.add_patch(p)

        
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        pax.set_title(color_field_name+" Seasonal Anom " +season_list[mon])

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    cb.set_label('Precip Anomaly mm/day')

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )
    
    plt.suptitle(str(case_name)+" "+color_field_name+" SST 850HGT WND seasonal anomaly")
    
    if save == 'yes':

        fig.savefig(target_dir+str(case_name)+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_anomaly.pdf")
        fig.savefig(target_dir+str(case_name)+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_anomaly.png")
        #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
        plt.close()# no need for dpi
    

    
def sst_plotting_function(color_field, contour_field, u_wnd, v_wnd, 
                          color_field_name, case_name, target_dir, *argv, save = 'yes'):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.015, top=0.94,
                         wspace=0.05, hspace=0.001)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    mm = range(9)

    for mon, pax in zip(mm, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax

        #reverse the coolwarm palatte to make blue rainy and red dry
        cmap = cm.coolwarm
        bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]


        x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
        im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                        extend='both')

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')


        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        
        #draw the box
        #lons = range(180,200,2.5)
        #lats = 
        #m.drawgreatcircle(180,20,180,0,linewidth = 2, color = 'b')
        
        if box_coords in argv:
            x1,y1 = m(box_coords[2], box_coords[1])
            x2,y2 = m(box_coords[2], box_coords[0])
            x3,y3 = m(box_coords[3], box_coords[0])
            x4,y4 = m(box_coords[3], box_coords[1])
            
            p = Polygon([(x1,y1), (x2,y2), (x3,y3), (x4,y4)], 
                        edgecolor='b', facecolor = 'none', linewidth = 2)
            m.ax.add_patch(p)
            #plt.gca().add_patch(p)

        
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        pax.set_title(color_field_name+" Seasonal Anom " +season_list[mon])

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    cb.set_label('SST Anomaly Deg C')

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )
    
    plt.suptitle(str(case_name)+" "+color_field_name+" SST 850HGT WND seasonal anomaly")
    
    if save == 'yes':
        fig.savefig(target_dir+str(case_name)+"_"+color_field_name+"_SST_850HGT_WND_seasonal_anomaly.pdf")
        fig.savefig(target_dir+str(case_name)+"_"+color_field_name+"_SST_850HGT_WND_seasonal_anomaly.png")
        #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
        plt.close()# no need for dpi

In [ ]:
box_coords =[]

year_list = [1975, 1981, 1984, 1986, 1989, 1996, 1997, 1999, 2000, 2001, 2002, 2006, 2008, 2009, 2012, 2013]

season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']

target_dir = "./total_rainfall_composite_member_maps/"

for year in year_list:

        sst_selection = ((sst_ca.ymdhms[:, 0] == year))
        sst_selection = np.nonzero(sst_selection)[0]
        sst_colorfield= sst_seasonal_anom.s_anom[sst_selection]

        PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year))
        PREC_precip_selection = np.nonzero(PREC_precip_selection)[0]
        PREC_precip_color_field = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]

        selection = ((uwnd_ca.ymdhms[:, 0] == year))
        selection = np.nonzero(selection)[0]
        uwnd= uwnd_seasonal_anom.s_anom[selection]
        vwnd = vwnd_seasonal_anom.s_anom[selection]
        hgt_field = hgt_seasonal_anom.s_anom[selection]

        plotting_function(PREC_precip_color_field, hgt_field, uwnd, vwnd, 
                          'PREC', year, target_dir, save = 'yes')
        
        sst_plotting_function(sst_colorfield, hgt_field, uwnd, vwnd, 
                             'ERSST', year, target_dir, save = 'yes')


        if year >=1979:
            precip_selection = ((precip_ca.ymdhms[:, 0] == year))
            precip_selection = np.nonzero(precip_selection)[0]
            precip_color_field= precip_seasonal_anom.s_anom[precip_selection]
            
            plotting_function(precip_color_field, hgt_field, uwnd, vwnd, 
                             'GPCP', year, target_dir, save = 'yes')

# Stop Here ^^^^^^^^^^^^^

In [ ]:
year = 1984

target_dir = "./composite_member_maps/"

sst_selection = ((sst_ca.ymdhms[:, 0] == year))
sst_selection = np.nonzero(sst_selection)[0]
sst_colorfield= sst_seasonal_anom.s_anom[sst_selection]

PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year))
PREC_precip_selection = np.nonzero(PREC_precip_selection)[0]
PREC_precip_color_field = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]

selection = ((uwnd_ca.ymdhms[:, 0] == year))
selection = np.nonzero(selection)[0]
uwnd= uwnd_seasonal_anom.s_anom[selection]
vwnd = vwnd_seasonal_anom.s_anom[selection]
hgt_field = hgt_seasonal_anom.s_anom[selection]

plotting_function(PREC_precip_color_field, hgt_field, 
                 uwnd, vwnd, 
                 'PREC', year,
                 target_dir)

In [ ]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader('.'))
template = env.get_template("table_template.html")

#template_vars = {"title" : "Sales Funnel Report - National",
#                 "national_pivot_table": ONI_df_color_back.to_html()}

#html_out = template.render(template_vars)

from weasyprint import HTML
#HTML(string=html_out).write_pdf("report.pdf")

#HTML(string=ONI_df_color_back.render()).write_pdf("test.pdf")

In [ ]:
from xhtml2pdf import pisa             # this is the module that will do the work
from matplotlib.backends.backend_pdf import PdfPages

# Utility function
def convertHtmlToPdf(sourceHtml, outputFilename):
    # open output file for writing (truncated binary)
    resultFile = open(outputFilename, "w+b")

    # convert HTML to PDF
    pisaStatus = pisa.CreatePDF(
            sourceHtml,                # the HTML to convert
            dest=resultFile,           # file handle to recieve result
            path='.')                  # this path is needed so relative paths for 
                                       # temporary image sources work

    # close output file
    resultFile.close()                 # close output file

    # return True on success and False on errors
    return pisaStatus.err

# Define your data
    sourceHtml = '<html><head>'         
    # add some table CSS in head
    sourceHtml += '''<style>
                     table, td, th {
                           border-style: double;
                           border-width: 3px;
                     }

                     td,th {
                           padding: 5px;
                     }
                     </style>'''
    sourceHtml += '</head><body>'

    # Add the dataframe
    sourceHtml += '\n<p>' + ONI_df_color_back.to_html() + '</p>'


    sourceHtml += '</body></html>'
    outputFilename = 'test.pdf'

    convertHtmlToPdf(sourceHtml, outputFilename)

In [ ]:
colors = plt.cm.BuPu(np.linspace(0, 0.5, 5))
colors
#ONI_df_color_back.to_excel('test.xlsx')

test = np.where(cond_verydry_cool[:-1,:], 'yellow', '')
#test

In [ ]:
print(type(ONI_df_color_back))

f = open('test.html', 'w')

f.write(ONI_df_color_back.render())

In [ ]:
from weasyprint import HTML
HTML(string=ONI_df_color_back.render()).write_pdf("test.pdf")

In [ ]:
from pandas.tools.plotting import table

ax = plt.subplot(111, frame_on = False)
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

table(ax, ONI_df)
plt.savefig('test.png')

In [ ]:
def test_color_cells_dry(data,cond_matrix = cond_dry_cool[:-1,:] ,color = 'yellow'):
    
    attr = 'background-color: {}'.format(color)
    
    return pd.DataFrame(np.where(cond_matrix, attr, ''),
                       index = data.index,
                       columns = data.columns)

test = test_color_cells_dry(ONI_df_color)
#ONI_df_color_back = ONI_df_color.style.apply(test_color_cells_dry, axis = None)
#print(type(test))
#print(test)
#test

In [ ]:
#Put ONI data into a pandas table
season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']



def color_cells(val):
    
    background_color = 'yellow' if cond_dry_cool else ''
    background_color = 'red' if cond_verydry_cool else ''
    
    return 'background-color: %s' % background_color 


ONI_df = pd.DataFrame(oni_period_matrix[:-1,], columns = season_list, index = range(1966,2016))
ONI_df_color = ONI_df.style.applymap(color_cells)
#ONI_df

#heat_map = np.empty(np.shape(oni_period_matrix[:-1,]), dtype = 'object')
#heat_map = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
#heat_map[:] = np.NAN

#if cond_dry_cool[:-1,:]:
heat_map_d = cond_dry_cool[:-1,:].astype(int)
heat_map_vd = cond_verydry_cool[:-1,:].astype(int)

heat_map_all = heat_map_d + heat_map_vd

#for dry, very_dry, value in np.nditer([cond_dry_cool[:-1,:],cond_verydry_cool[:-1,:],heat_map], flags=['refs_ok']):
    #print(dry, very_dry, value)
    
#    if dry:
#        heat_map(value) = 'yellow' 
        #print(dry, value)
#    if very_dry:
#        vheat_map(value) = 'red'
        #print(very_dry, value)
    
fig,axs = plt.subplots(2,1)
#ax = fig.add_subplot(111,frameon =True, xticks=[], yticks=[])

the_table = axs[0].table(cellText = ONI_df.values, 
                      rowLabels=ONI_df.index, colLabels = ONI_df.columns,
                      cellColours = plt.cm.hot(heat_map_all) )




#print(heat_map_all)

In [ ]:
selection_matrix = cond_verydry_cool

for i in range(len(selection_matrix.ravel())):
    if selection_matrix.ravel()[i]:
        print(stations.Kwajalein.ymdhms[i])

In [ ]:
selection_matrix = cond_verydry_cool_90patcor

for i in range(len(selection_matrix.ravel())):
    if selection_matrix.ravel()[i]:
        print(stations.Kwajalein.ymdhms[i])
    

# Below we set up the matrices that will contain the composite members

In [ ]:
season_members_verydry = np.zeros(12)
gpcp_season_members_verydry = np.zeros(12)

sst_composite_members_verydry = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_verydry[:] = np.NAN

precip_composite_members_verydry = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_verydry[:] = np.NAN

PREC_precip_composite_members_verydry = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_verydry[:] = np.NAN

uwnd_composite_members_verydry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_verydry[:] = np.NAN
vwnd_composite_members_verydry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_verydry[:] = np.NAN
hgt_composite_members_verydry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_verydry[:] = np.NAN

season_members_dry = np.zeros(12)
gpcp_season_members_dry = np.zeros(12)

precip_composite_members_dry = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_dry[:] = np.NAN

sst_composite_members_dry = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_dry[:] = np.NAN

PREC_precip_composite_members_dry = np.empty([51,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_dry[:] = np.NAN

uwnd_composite_members_dry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_dry[:] = np.NAN
vwnd_composite_members_dry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_dry[:] = np.NAN
hgt_composite_members_dry = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_dry[:] = np.NAN

In [ ]:
season_members_verydry_90patcor = np.zeros(12)
gpcp_season_members_verydry_90patcor = np.zeros(12)

precip_composite_members_verydry_90patcor = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_verydry_90patcor[:] = np.NAN

sst_composite_members_verydry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_verydry_90patcor[:] = np.NAN

PREC_precip_composite_members_verydry_90patcor = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_verydry_90patcor[:] = np.NAN

uwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_verydry_90patcor[:] = np.NAN
vwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_verydry_90patcor[:] = np.NAN
hgt_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_verydry_90patcor[:] = np.NAN

season_members_dry_90patcor = np.zeros(12)
gpcp_season_members_dry_90patcor = np.zeros(12)

precip_composite_members_dry_90patcor = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_dry_90patcor[:] = np.NAN

sst_composite_members_dry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_dry_90patcor[:] = np.NAN

PREC_precip_composite_members_dry_90patcor = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_dry_90patcor[:] = np.NAN

uwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_composite_members_dry_90patcor[:] = np.NAN
vwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_composite_members_dry_90patcor[:] = np.NAN
hgt_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_composite_members_dry_90patcor[:] = np.NAN

In [ ]:
#season_members_verydry_90patcor = np.zeros(12)

#sst_composite_members_verydry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
#sst_composite_members_verydry_90patcor[:] = np.NAN
#uwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#uwnd_composite_members_verydry_90patcor[:] = np.NAN
#vwnd_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#vwnd_composite_members_verydry_90patcor[:] = np.NAN
#hgt_composite_members_verydry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#hgt_composite_members_verydry_90patcor[:] = np.NAN

#season_members_dry_90patcor = np.zeros(12)

#sst_composite_members_dry_90patcor = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
#sst_composite_members_dry_90patcor[:] = np.NAN
#uwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#uwnd_composite_members_dry_90patcor[:] = np.NAN
#vwnd_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#vwnd_composite_members_dry_90patcor[:] = np.NAN
#hgt_composite_members_dry_90patcor = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
#hgt_composite_members_dry_90patcor[:] = np.NAN

In [ ]:
year_range = range(1966,2016)
for year in year_range:
    for m in range(1,13):
        
        if cond_verydry_cool[year_range.index(year),m-1]:   
            
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_verydry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_verydry[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_verydry[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_verydry[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_verydry[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_verydry[m-1] = season_members_verydry[m-1] +1
            
            if year >=1979:
                precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
                precip_selection = np.nonzero(precip_selection)[0][0]
                precip_composite_members_verydry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
                       
                gpcp_season_members_verydry[m-1] = gpcp_season_members_verydry[m-1] +1
                

#--------------------------------------------------------------------------------------------------------------------
for year in year_range:

    for m in range(1,13):
        
        if cond_verydry_cool_90patcor[year_range.index(year),m-1]:   
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_verydry_90patcor[m-1] = season_members_verydry_90patcor[m-1] +1
            
            if year >=1979:
                precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
                precip_selection = np.nonzero(precip_selection)[0][0]
                precip_composite_members_verydry_90patcor[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
                       
                gpcp_season_members_verydry_90patcor[m-1] = gpcp_season_members_verydry_90patcor[m-1] +1

In [ ]:
#print(np.shape(PREC_precip_seasonal_anom.s_anom))
#PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == 1999) & (PREC_precip_ca.ymdhms[:, 1] == m))
#PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
#print(PREC_precip_seasonal_anom.s_anom[PREC_precip_selection])

#year_range = range(1966,2016)
#for year in year_range:
#    print(year_range.index(year), year)

In [ ]:
year_range = range(1966,2016)
for year in year_range:
    for m in range(1,13):
        
        if cond_dry_cool[year_range.index(year),m-1]:   
            
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_dry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_dry[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_dry[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_dry[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_dry[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_dry[m-1] = season_members_dry[m-1] +1
            
            if year >=1979:
                precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
                precip_selection = np.nonzero(precip_selection)[0][0]
                precip_composite_members_dry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
                       
                gpcp_season_members_dry[m-1] = gpcp_season_members_dry[m-1] +1
                

#--------------------------------------------------------------------------------------------------------------------
for year in year_range:

    for m in range(1,13):
        
        if cond_dry_cool_90patcor[year_range.index(year),m-1]:   
            
            #print(year,year_range.index(year),m,cond_verydry_cool[year_range.index(year),m-1])

            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_dry_90patcor[m-1] = season_members_dry_90patcor[m-1] +1
            
            if year >=1979:
                precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
                precip_selection = np.nonzero(precip_selection)[0][0]
                precip_composite_members_dry_90patcor[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
                       
                gpcp_season_members_dry_90patcor[m-1] = gpcp_season_members_dry_90patcor[m-1] +1



In [ ]:
precip_composite_verydry = np.nanmean(precip_composite_members_verydry, axis=0)
sst_composite_verydry = np.nanmean(sst_composite_members_verydry, axis=0)
PREC_precip_composite_verydry = np.nanmean(PREC_precip_composite_members_verydry, axis=0)
uwnd_composite_verydry = np.nanmean(uwnd_composite_members_verydry, axis=0)
vwnd_composite_verydry = np.nanmean(vwnd_composite_members_verydry, axis=0)
hgt_composite_verydry = np.nanmean(hgt_composite_members_verydry, axis=0)

precip_composite_dry = np.nanmean(precip_composite_members_dry, axis=0)
sst_composite_dry = np.nanmean(sst_composite_members_dry, axis=0)
PREC_precip_composite_dry  = np.nanmean(PREC_precip_composite_members_dry, axis=0)
uwnd_composite_dry = np.nanmean(uwnd_composite_members_dry, axis=0)
vwnd_composite_dry = np.nanmean(vwnd_composite_members_dry, axis=0)
hgt_composite_dry = np.nanmean(hgt_composite_members_dry, axis=0)

In [ ]:
precip_composite_verydry_90patcor = np.nanmean(precip_composite_members_verydry_90patcor, axis=0)
sst_composite_verydry_90patcor = np.nanmean(sst_composite_members_verydry_90patcor, axis=0)
PREC_precip_composite_verydry_90patcor = np.nanmean(PREC_precip_composite_members_verydry_90patcor, axis=0)
uwnd_composite_verydry_90patcor = np.nanmean(uwnd_composite_members_verydry_90patcor, axis=0)
vwnd_composite_verydry_90patcor = np.nanmean(vwnd_composite_members_verydry_90patcor, axis=0)
hgt_composite_verydry_90patcor = np.nanmean(hgt_composite_members_verydry_90patcor, axis=0)

precip_composite_dry_90patcor = np.nanmean(precip_composite_members_dry_90patcor, axis=0)
sst_composite_dry_90patcor = np.nanmean(sst_composite_members_dry_90patcor, axis=0)
PREC_precip_composite_dry_90patcor = np.nanmean(PREC_precip_composite_members_dry_90patcor, axis=0)
uwnd_composite_dry_90patcor = np.nanmean(uwnd_composite_members_dry_90patcor, axis=0)
vwnd_composite_dry_90patcor = np.nanmean(vwnd_composite_members_dry_90patcor, axis=0)
hgt_composite_dry_90patcor = np.nanmean(hgt_composite_members_dry_90patcor, axis=0)

In [ ]:
season_list = ['DJF' , 'JFM' , 'FMA',
               'MAM' , 'AMJ' , 'MJJ',
               'JJA' , 'JAS' , 'ASO',
               'SON' , 'OND' , 'NDJ']

month_list = ['Jan' , 'Feb' , 'Mar',
              'Apr' , 'May' , 'Jun',
              'Jul' , 'Aug' , 'Sep',
              'Oct' , 'Nov' , 'Dec']


plt.rcParams['figure.dpi'] = 113
plt.rcParams['axes.titlesize'] = 'small'
plt.rcParams['ytick.labelsize'] = 'small'  # for colorbar

# Here I define the plotting function

In [ ]:
def plotting_function(color_field, contour_field, u_wnd, v_wnd, member_counter, color_field_name, case_name):    
    
    #here basemap is producing a strange behaviour
    m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                        llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

    with open('mapcache.pk', mode='wb') as f:
        pickle.dump(m, f)

    ## Tweak the subplot specifications.  

    subplotparams = dict(left=0.03, right=0.88,
                         bottom=0.03, top=0.96,
                         wspace=0.05, hspace=0.1)


    fig, axs = plt.subplots(3,3, #sharex=True,
                        figsize=(13, 7.8),
                        gridspec_kw=subplotparams,
                       )

    mm = range(2,11)

    for mon, pax in zip(mm, axs.flat):

        with open('mapcache.pk', 'rb') as f:
            m = pickle.load(f)
        m.ax = pax

        #reverse the coolwarm palatte to make blue rainy and red dry
        cmap = plt.get_cmap('BrBG')
        bounds = [-2.5,-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2, 2.5]


        x, y = m(*np.meshgrid(PREC_precip_longitudes, PREC_precip_latitudes))
        im = m.contourf(x, y, color_field[mon,:,:], levels=bounds, cmap=cmap,
                        extend='both')

        x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
        #hgt_bounds = np.arange(-40,40,1)
        hgt_bounds = [-80, -70,-60, -50, -40, 
                      -30, -20, -10,-8, -6, 
                      -4, -2, -1,0, 1, 2, 4, 
                      6, 8, 10, 20, 30,
                      40, 50, 60, 70, 80]
        hgt_bound_0 = [0]
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bounds,
                           linewidths=0.5, colors='k')
        im_hgt = m.contour(x_hgt, y_hgt, contour_field[mon,:,:], levels=hgt_bound_0,
                       linewidths=0.9, colors='k')


        # transform vectors to projection grid.
        uproj, vproj, xx, yy = m.rotate_vector(u_wnd[mon,:,:], 
                                               v_wnd[mon,:,:], 
                                               ncep_longitudes, 
                                               ncep_latitudes,
                                               returnxy=True)
        # now plot every other vector
        Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                     uproj[::2,::2], vproj[::2,::2],
                     scale=20, scale_units='inches')

        m.drawcoastlines()
        parallels = np.arange(-90, 90, 30)
        meridians = np.arange(-180, 180, 60)
        if pax in axs.flat[::3]:
            m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        else:
            m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

        if pax in axs.flat[:6]:
            m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
        if pax in axs.flat[6:]:
            m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
        #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

        pax.set_title(color_field_name+" Seasonal Anom Comp " +season_list[mon] + " members " + str(member_counter[mon]))

    #cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
    ##  This method of making an Axes for the cbar interacts very badly with
    ##  basemap, so just make one manually.  This also gives more control, so
    ##  it looks nicer.
    left = subplotparams['right'] + 0.02
    bottom = subplotparams['bottom'] + 0.05
    width = 0.015
    height = subplotparams['top'] - subplotparams['bottom'] - 0.1

    cax = fig.add_axes([left, bottom, width, height])
    cb = plt.colorbar(im, cax=cax)
    cb.set_label('Precip Anomaly mm/day')

    ## EF: The quiverkey needs to be made using an axes in which a quiver is
    ##     drawn so that it has the right transform information.  Otherwise
    ##     it won't get the length right.
    qkx = left + (1 - left) / 4
    qky = bottom + height + 0.025
    qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                       coordinates='figure',
                       labelpos='N',
                       labelsep=0.07, 
                       fontproperties=dict(size='small'),
                      )

    fig.savefig(case_name+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_anomaly.pdf")
    fig.savefig(case_name+"_"+color_field_name+"_PRECIP_850HGT_WND_seasonal_anomaly.png")
    #fig.savefig("VERY_DRY_COMPOSITE_SST_850HGT_WND_seasonal_anomaly_%04d-%02d.pdf" % (year, mon))
    plt.close()# no need for dpi

In [ ]:
#plotting_function(PREC_precip_composite_verydry, hgt_composite_verydry, 
#                  uwnd_composite_verydry, vwnd_composite_verydry, 
#                  season_members_verydry, 'PREC', "Very_Dry")

# Here we plot the composites for the verydry cases

## verydry case composite with PREC data

In [ ]:
plotting_function(PREC_precip_composite_verydry, hgt_composite_verydry, 
                  uwnd_composite_verydry, vwnd_composite_verydry, 
                  season_members_verydry, 'PREC', "Very_Dry")

plotting_function(PREC_precip_composite_verydry_90patcor, hgt_composite_verydry_90patcor, 
                  uwnd_composite_verydry_90patcor, vwnd_composite_verydry_90patcor, 
                  season_members_verydry_90patcor, 'PREC', "Very_Dry_90patcor")

plotting_function(PREC_precip_composite_dry, hgt_composite_dry, 
                  uwnd_composite_dry, vwnd_composite_dry, 
                  season_members_dry, 'PREC', "Dry")

plotting_function(PREC_precip_composite_dry_90patcor, hgt_composite_dry_90patcor, 
                  uwnd_composite_dry_90patcor, vwnd_composite_dry_90patcor, 
                  season_members_dry_90patcor, 'PREC', "Dry_90patcor")

## GPCP verydry 90 patcor composites

In [ ]:
plotting_function(precip_composite_verydry, hgt_composite_verydry, 
                  uwnd_composite_verydry, vwnd_composite_verydry, 
                  gpcp_season_members_verydry, 'GPCP', "Very_Dry")


plotting_function(precip_composite_verydry_90patcor, hgt_composite_verydry_90patcor, 
                  uwnd_composite_verydry_90patcor, vwnd_composite_verydry_90patcor, 
                  gpcp_season_members_verydry_90patcor, 'GPCP', "Very_Dry_90patcor")

plotting_function(precip_composite_dry, hgt_composite_dry, 
                  uwnd_composite_dry, vwnd_composite_dry, 
                  gpcp_season_members_dry, 'GPCP', "Dry")


plotting_function(precip_composite_dry_90patcor, hgt_composite_dry_90patcor, 
                  uwnd_composite_dry_90patcor, vwnd_composite_dry_90patcor, 
                  gpcp_season_members_dry_90patcor, 'GPCP', "Dry_90patcor")

## SST maps

In [ ]:
sst_plotting_function(sst_composite_verydry, hgt_composite_verydry, 
                      uwnd_composite_verydry, vwnd_composite_verydry, 
                      season_members_verydry, 'ERSST', "Very_Dry")


sst_plotting_function(sst_composite_verydry_90patcor, hgt_composite_verydry_90patcor, 
                      uwnd_composite_verydry_90patcor, vwnd_composite_verydry_90patcor, 
                      season_members_verydry_90patcor, 'ERSST', "Very_Dry_90patcor")

sst_plotting_function(sst_composite_dry, hgt_composite_dry, 
                      uwnd_composite_dry, vwnd_composite_dry, 
                      season_members_dry, 'ERSST', "Dry")


sst_plotting_function(sst_composite_dry_90patcor, hgt_composite_dry_90patcor, 
                      uwnd_composite_dry_90patcor, vwnd_composite_dry_90patcor, 
                      season_members_dry_90patcor, 'ERSST', "Dry_90patcor")

# Make the non dry composites

In [ ]:
cond_non_dry = kwajalein_seasonal_anom_matrix >= 0* kwajalein_season_std_matrix
cond_oni = oni_period_matrix <= -0.1
cond_non_dry_cool = np.logical_and(cond_non_dry, cond_oni)

precip_composite_members_non_dry = np.empty([50,12, len(precip_latitudes), len(precip_longitudes)])
precip_composite_members_non_dry[:] = np.NAN

sst_composite_members_non_dry = np.empty([50,12, len(ersst_latitudes), len(ersst_longitudes)])
sst_composite_members_non_dry[:] = np.NAN

PREC_precip_composite_members_non_dry = np.empty([50,12, len(PREC_precip_latitudes), len(PREC_precip_longitudes)])
PREC_precip_composite_members_non_dry[:] = np.NAN

uwnd_non_dry_composite_members = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
uwnd_non_dry_composite_members[:] = np.NAN
vwnd_non_dry_composite_members = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
vwnd_non_dry_composite_members[:] = np.NAN
hgt_non_dry_composite_members = np.empty([50,12,len(ncep_latitudes), len(ncep_longitudes)])
hgt_non_dry_composite_members[:] = np.NAN

gpcp_season_members_non_dry = np.zeros(12)
season_members_non_dry = np.zeros(12)

In [ ]:
year_range = range(1966,2016)
for year in year_range:
    for m in range(1,13):
        
        if cond_non_dry_cool[year_range.index(year),m-1]:
            
            sst_selection = ((sst_ca.ymdhms[:, 0] == year) & (sst_ca.ymdhms[:, 1] == m))
            sst_selection = np.nonzero(sst_selection)[0][0]
            sst_composite_members_non_dry[year_range.index(year),m-1,:,:] = sst_seasonal_anom.s_anom[sst_selection]
            
            PREC_precip_selection = ((PREC_precip_ca.ymdhms[:, 0] == year) & (PREC_precip_ca.ymdhms[:, 1] == m))
            PREC_precip_selection = np.nonzero(PREC_precip_selection)[0][0]
            PREC_precip_composite_members_non_dry[year_range.index(year),m-1,:,:] = PREC_precip_seasonal_anom.s_anom[PREC_precip_selection]
            
            selection = ((uwnd_ca.ymdhms[:, 0] == year) & (uwnd_ca.ymdhms[:, 1] == m))
            selection = np.nonzero(selection)[0][0]
            uwnd_non_dry_composite_members[year_range.index(year),m-1,:,:] = uwnd_seasonal_anom.s_anom[selection]
            vwnd_non_dry_composite_members[year_range.index(year),m-1,:,:] = vwnd_seasonal_anom.s_anom[selection]
            hgt_non_dry_composite_members[year_range.index(year),m-1,:,:] = hgt_seasonal_anom.s_anom[selection]
            
            season_members_non_dry[m-1] = season_members_non_dry[m-1] +1
            
            if year >=1979:
                precip_selection = ((precip_ca.ymdhms[:, 0] == year) & (precip_ca.ymdhms[:, 1] == m))
                precip_selection = np.nonzero(precip_selection)[0][0]
                precip_composite_members_non_dry[year_range.index(year),m-1,:,:] = precip_seasonal_anom.s_anom[precip_selection]
                       
                gpcp_season_members_non_dry[m-1] = gpcp_season_members_non_dry[m-1] +1

In [ ]:
precip_non_dry_composite = np.nanmean(precip_composite_members_non_dry, axis=0)

sst_non_dry_composite = np.nanmean(sst_composite_members_non_dry, axis=0)

PREC_precip_non_dry_composite = np.nanmean(PREC_precip_composite_members_non_dry, axis=0)

uwnd_non_dry_composite = np.nanmean(uwnd_non_dry_composite_members, axis=0)
vwnd_non_dry_composite = np.nanmean(vwnd_non_dry_composite_members, axis=0)
hgt_non_dry_composite = np.nanmean(hgt_non_dry_composite_members, axis=0)

In [ ]:
print(np.shape(uwnd_non_dry_composite))
print(np.shape(vwnd_non_dry_composite))
print(np.shape(hgt_non_dry_composite))

print(np.shape(precip_non_dry_composite))
print(np.shape(PREC_precip_non_dry_composite))

In [ ]:
plotting_function(PREC_precip_non_dry_composite, hgt_non_dry_composite, 
                  uwnd_non_dry_composite, vwnd_non_dry_composite, 
                  season_members_non_dry, 'PREC', "Non_Dry")

plotting_function(precip_non_dry_composite, hgt_non_dry_composite, 
                  uwnd_non_dry_composite, vwnd_non_dry_composite, 
                  gpcp_season_members_non_dry, 'GPCP', "Non_Dry")

sst_plotting_function(sst_non_dry_composite, hgt_non_dry_composite, 
                      uwnd_non_dry_composite, vwnd_non_dry_composite, 
                      season_members_non_dry, 'ERSST', "Non_Dry")

# Stop Here-------------------------^

In [ ]:
#here basemap is producing a strange behaviour
m = Basemap(projection='merc', llcrnrlat=-50.1, urcrnrlat=50.01,
                    llcrnrlon=80, urcrnrlon=300, lat_ts=0, resolution='c')

with open('mapcache.pk', mode='wb') as f:
    pickle.dump(m, f)

## Tweak the subplot specifications.  

subplotparams = dict(left=0.03, right=0.88,
                     bottom=0.03, top=0.96,
                     wspace=0.05, hspace=0.1)

    
fig, axs = plt.subplots(3,3, #sharex=True,
                    figsize=(13, 7.8),
                    gridspec_kw=subplotparams,
                   )

mm = range(2,11)

for mon, pax in zip(mm, axs.flat):

    with open('mapcache.pk', 'rb') as f:
        m = pickle.load(f)
    m.ax = pax

    #reverse the coolwarm palatte to make blue rainy and red dry
    cmap = cm.coolwarm
    bounds = [-2, -1.5,-1.25, -1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1, 1.25, 1.5, 2]
    #norm = cols.BoundaryNorm(bounds, cmap.N)

    #im = m.imshow(sst_transform,cmap=cm.coolwarm, interpolation = "none",norm=norm)

    ##  EF: using an image gains nothing here, loses resolution, and adds complexity.
    ##      Contouring is nicer.  If you wanted something image-like, it would
    ##      be better to use pcolormesh--but there is no point when you are using
    ##      discretized colors on a relatively fine lon/lat grid like this.

    x, y = m(*np.meshgrid(ersst_longitudes, ersst_latitudes))
    im = m.contourf(x, y, sst_non_dry_composite[mon,:,:], levels=bounds, cmap=cmap,
                    extend='both')

    x_hgt, y_hgt = m(*np.meshgrid(ncep_longitudes, ncep_latitudes))
    #hgt_bounds = np.arange(-40,40,1)
    hgt_bounds = [-80, -70,-60, -50, -40, 
                  -30, -20, -10,-8, -6, 
                  -4, -2, -1,0, 1, 2, 4, 
                  6, 8, 10, 20, 30,
                  40, 50, 60, 70, 80]
    im_hgt = m.contour(x_hgt, y_hgt, hgt_non_dry_composite[mon,:,:], levels=hgt_bounds,
                       linewidths=0.5, colors='k')


    # transform vectors to projection grid.
    uproj, vproj, xx, yy = m.rotate_vector(uwnd_non_dry_composite[mon,:,:], 
                                           vwnd_non_dry_composite[mon,:,:], 
                                           ncep_longitudes, 
                                           ncep_latitudes,
                                           returnxy=True)
    # now plot every other vector
    Q = m.quiver(xx[::2,::2], yy[::2,::2], 
                 uproj[::2,::2], vproj[::2,::2],
                 scale=20, scale_units='inches')

    m.drawcoastlines()
    parallels = np.arange(-90, 90, 30)
    meridians = np.arange(-180, 180, 60)
    if pax in axs.flat[::3]:
        m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
    else:
        m.drawparallels(parallels, labels = [0, 0, 0, 0], fontsize=8)

    if pax in axs.flat[:6]:
        m.drawmeridians(meridians, labels = [0, 0, 0, 0], fontsize=8)
    if pax in axs.flat[6:]:
        m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

    #m.drawparallels(parallels, labels = [1, 0, 0, 1], fontsize=8)
    #m.drawmeridians(meridians, labels = [1, 0, 0, 1], fontsize=8)

    pax.set_title("Seasonal Anomaly Composite " +season_list[mon] + " members " + str(season_members_non_dry[mon]))

#cax, kw = mpl.colorbar.make_axes([ax for ax in axs.flat])
##  This method of making an Axes for the cbar interacts very badly with
##  basemap, so just make one manually.  This also gives more control, so
##  it looks nicer.
left = subplotparams['right'] + 0.02
bottom = subplotparams['bottom'] + 0.05
width = 0.015
height = subplotparams['top'] - subplotparams['bottom'] - 0.1

cax = fig.add_axes([left, bottom, width, height])
cb = plt.colorbar(im, cax=cax)
cb.set_label('SST C')

## EF: The quiverkey needs to be made using an axes in which a quiver is
##     drawn so that it has the right transform information.  Otherwise
##     it won't get the length right.
qkx = left + (1 - left) / 4
qky = bottom + height + 0.025
qk = pax.quiverkey(Q, qkx, qky, 10, '10 m/s', 
                   coordinates='figure',
                   labelpos='N',
                   labelsep=0.07, 
                   fontproperties=dict(size='small'),
                  )

fig.savefig("NON_DRY_COMPOSITE_SST_850HGT_WNDseasonal_anomaly.pdf")
#plt.close()# no need for dpi 